<a href="https://colab.research.google.com/github/Harshi-63/AD-Lab/blob/main/Lab-2/CatDogModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

100%|██████████| 1.06G/1.06G [00:16<00:00, 70.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1


In [7]:
import os
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.model_selection import train_test_split


In [1]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import image_dataset_from_directory

# Define paths for training and testing datasets
train_dir = "/root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1/train"
test_dir = "/root/.cache/kagglehub/datasets/salader/dogs-vs-cats/versions/1/test"

# Function to normalize the images (on-the-fly)
def normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    return image, label

# Create the training dataset
train_ds = image_dataset_from_directory(
    train_dir,
    labels="inferred",
    label_mode="categorical",  # One-hot encoding of labels
    batch_size=32,
    image_size=(64, 64),       # Resize all images to 64x64
    shuffle=True,              # Shuffle the dataset
    seed=123                   # Set a seed for reproducibility
)

# Apply normalization to the training dataset on-the-fly
train_ds = train_ds.map(normalize_image)

# Create the test dataset
test_ds = image_dataset_from_directory(
    test_dir,
    labels="inferred",
    label_mode="categorical",  # One-hot encoding of labels
    batch_size=32,
    image_size=(64, 64),
    shuffle=False              # Do not shuffle the test set
)

# Apply normalization to the test dataset on-the-fly
test_ds = test_ds.map(normalize_image)

# Convert the training dataset to numpy arrays (Optional: For further processing outside TensorFlow)
# We will now iterate through the dataset and store batches in numpy arrays
X_train, y_train = [], []
for batch, labels in train_ds:
    X_train.append(batch.numpy())
    y_train.append(labels.numpy())

X_train = np.concatenate(X_train, axis=0)  # Concatenate all batches into one array
y_train = np.concatenate(y_train, axis=0)  # Concatenate all label batches

# Convert one-hot labels to integer labels
y_train_labels = np.argmax(y_train, axis=1)

# Normalize and flatten images for traditional ML models (Optional)
X_train_flat = X_train.reshape(X_train.shape[0], -1)  # Flatten the images
X_train_flat /= 255.0  # Normalize pixel values to [0, 1]

# Split the dataset into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(
    X_train_flat, y_train_labels, test_size=0.2, random_state=42
)

# Now X_train_split, y_train_split can be used to train ML models like SVM, Random Forest, etc.
# And X_test_split, y_test_split can be used for evaluation




Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize Logistic Regression
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
log_reg_model.fit(X_train_split, y_train_split)

# Make predictions
y_pred = log_reg_model.predict(X_test_split)

# Evaluate accuracy
log_reg_accuracy = accuracy_score(y_test_split, y_pred)
print(f"Logistic Regression Accuracy: {log_reg_accuracy:.4f}")


Logistic Regression Accuracy: 0.5563


In [4]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_split, y_train_split)

# Make predictions
y_pred_rf = rf_model.predict(X_test_split)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test_split, y_pred_rf)
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")


Random Forest Accuracy: 0.6558


In [2]:
from sklearn.linear_model import SGDClassifier

# Faster version of SVM using Stochastic Gradient Descent
svm_sgd = SGDClassifier(loss="hinge", alpha=0.0001, max_iter=1000, tol=1e-3, random_state=42)

# Train model
svm_sgd.fit(X_train_split, y_train_split)

# Evaluate
svm_sgd_accuracy = svm_sgd.score(X_test_split, y_test_split)
print(f"SVM (SGD) Accuracy: {svm_sgd_accuracy:.4f}")


SVM (SGD) Accuracy: 0.5547


In [6]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, accuracy_score

# Initialize K-means
kmeans_model = KMeans(n_clusters=2, random_state=42, n_init=10)

# Train the model (fit K-means to the data)
kmeans_model.fit(X_train_split)

# Get the predicted clusters (K-means is unsupervised, so clusters are arbitrary)
y_pred_kmeans = kmeans_model.predict(X_test_split)

# Adjust cluster labels to match actual labels (since K-means doesn't assign specific labels)
# This is done by checking which cluster corresponds better to "Dog" or "Cat"
from scipy.stats import mode

def match_labels(y_true, y_pred):
    labels = np.zeros_like(y_pred)
    for i in range(2):  # We have two clusters (dogs vs cats)
        mask = (y_pred == i)
        labels[mask] = mode(y_true[mask])[0]
    return labels

# Adjust predicted labels to match ground truth
y_pred_kmeans_adjusted = match_labels(y_test_split, y_pred_kmeans)

# Compute accuracy
kmeans_accuracy = accuracy_score(y_test_split, y_pred_kmeans_adjusted)
print(f"K-means Clustering Accuracy: {kmeans_accuracy:.4f}")

# Compute Adjusted Rand Index (better for clustering evaluation)
kmeans_ari = adjusted_rand_score(y_test_split, y_pred_kmeans)
print(f"K-means Clustering Adjusted Rand Index: {kmeans_ari:.4f}")


K-means Clustering Accuracy: 0.5215
K-means Clustering Adjusted Rand Index: 0.0016


In [9]:
import joblib

# Save models
joblib.dump(svm_sgd, "svm_model.pkl")
joblib.dump(rf_model, "random_forest_model.pkl")
joblib.dump(log_reg_model, "log_reg_model.pkl")
joblib.dump(kmeans_model, "kmeans_model.pkl")

print("Models saved successfully!")


Models saved successfully!
